In [ ]:
!pip install plotly==5.3.1

In [ ]:
!pip install SimpleITK

In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import SimpleITK as sitk
import plotly.express as px
import plotly.graph_objects as go
import math

In [2]:
def load_img_from_tiff(path2img):
    img = sitk.ReadImage(path2img)
    img_array = sitk.GetArrayFromImage(img)
    return(img_array)

In [ ]:
path = "/content/drive/MyDrive/mydata/sea_urchin_data/3D/Galleria Piastra Modello 1 100x100 Echi1-10x.tif"
data = load_img_from_tiff(path)
px.imshow(data[0])

In [ ]:
fig = px.imshow(data, animation_frame=0, binary_string=True, labels=dict(animation_frame="slice"))
fig.show()